<a href="https://colab.research.google.com/github/abhilashgn/CNTK/blob/master/Lyft_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#to access kaggle datasets
!pip install kaggle
#Math operations
!pip install numpy==1.15.0
#Machine learning
!pip install catboost

     |████████████████████████████████| 66.1MB 60kB/s 


In [2]:
#data preprocessing
import pandas as pd
#math operations
import numpy as np
#machine learning
from catboost import CatBoostRegressor, Pool
#data scaling
from sklearn.preprocessing import StandardScaler
#hyperparameter optimization
from sklearn.model_selection import GridSearchCV
#support vector machine model
from sklearn.svm import NuSVR, SVR
#kernel ridge model
from sklearn.kernel_ridge import KernelRidge
#data visualization
import matplotlib.pyplot as plt


In [6]:
# Colab's file access feature
from google.colab import files

#retrieve uploaded file
uploaded = files.upload()

#print results
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/E:/ISB/Material/Learn/Lyft/

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes
chmod: cannot access '/root/E:/ISB/Material/Learn/Lyft/': No such file or directory


In [7]:
#list competitions
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        254           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         90           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        366           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2552           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      18980            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       4930           False  
connectx

In [9]:
#download Lyft Prediction data, takes time

!kaggle competitions download -c lyft-motion-prediction-autonomous-vehicles

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: force download


In [10]:
!ls

0     1006  103  112		     lyft1009.tfw  lyft1019.tfw
1     1007  104  113		     lyft1010.tfw  lyft1020.tfw
10    1008  105  114		     lyft1011.tfw  lyft1021.tfw
100   1009  106  12		     lyft1012.tfw  lyft1022.tfw
1000  101   107  aerial_map.png.zip  lyft1013.tfw  lyft1023.tfw
1001  1010  108  feedback.txt	     lyft1014.tfw  lyft1107.tfw
1002  1011  109  LICENSE	     lyft1015.tfw  lyft1108.tfw
1003  1012  11	 lyft1006.tfw	     lyft1016.tfw  sample_data
1004  1013  110  lyft1007.tfw	     lyft1017.tfw  semantic_map.pb.zip
1005  102   111  lyft1008.tfw	     lyft1018.tfw


In [12]:
!unzip semantic_map.pb.zip
!ls

Archive:  semantic_map.pb.zip
replace semantic_map.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: semantic_map.pb         
0     1006  103  112		     lyft1009.tfw  lyft1019.tfw
1     1007  104  113		     lyft1010.tfw  lyft1020.tfw
10    1008  105  114		     lyft1011.tfw  lyft1021.tfw
100   1009  106  12		     lyft1012.tfw  lyft1022.tfw
1000  101   107  aerial_map.png.zip  lyft1013.tfw  lyft1023.tfw
1001  1010  108  feedback.txt	     lyft1014.tfw  lyft1107.tfw
1002  1011  109  LICENSE	     lyft1015.tfw  lyft1108.tfw
1003  1012  11	 lyft1006.tfw	     lyft1016.tfw  sample_data
1004  1013  110  lyft1007.tfw	     lyft1017.tfw  semantic_map.pb
1005  102   111  lyft1008.tfw	     lyft1018.tfw  semantic_map.pb.zip


In [13]:
!ls

0     1006  103  112		     lyft1009.tfw  lyft1019.tfw
1     1007  104  113		     lyft1010.tfw  lyft1020.tfw
10    1008  105  114		     lyft1011.tfw  lyft1021.tfw
100   1009  106  12		     lyft1012.tfw  lyft1022.tfw
1000  101   107  aerial_map.png.zip  lyft1013.tfw  lyft1023.tfw
1001  1010  108  feedback.txt	     lyft1014.tfw  lyft1107.tfw
1002  1011  109  LICENSE	     lyft1015.tfw  lyft1108.tfw
1003  1012  11	 lyft1006.tfw	     lyft1016.tfw  sample_data
1004  1013  110  lyft1007.tfw	     lyft1017.tfw  semantic_map.pb
1005  102   111  lyft1008.tfw	     lyft1018.tfw  semantic_map.pb.zip


In [13]:
#Extract training data into a dataframe for further manipulation
train = pd.read_csv('sample_data', nrows=600000, dtype={'acoustic_data': np.int16,
                                                       'time_to_failure': np.float64})